In [1]:
import gradio as gr
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import cv2
from segment_anything import sam_model_registry
from segment_anything.predictor_sammed import SammedPredictor
from argparse import Namespace
import torch
import torchvision
import os, sys
import random
import warnings
from scipy import ndimage

device = "cpu" #torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
args = Namespace()
args.device = device
args.image_size = 256
args.encoder_adapter = True
args.sam_checkpoint = "pretrain_model/sam-med2d_b.pth"  #sam_vit_b.pth  sam-med2d_b.pth
def load_model(args):
    model = sam_model_registry["vit_b"](args).to(args.device)
    model.eval()
    predictor = SammedPredictor(model)
    return predictor
    
predictor = load_model(args)

e:\Anacoda\envs\MMseg\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu
*******load pretrain_model/sam-med2d_b.pth


In [2]:

def draw_mask(mask, draw, random_color=False):
    if random_color:
        color = (random.randint(0, 255), random.randint(
            0, 255), random.randint(0, 255), 153)
    else:
        color = (30, 144, 255, 153)

    nonzero_coords = np.transpose(np.nonzero(mask))

    for coord in nonzero_coords:
        draw.point(coord[::-1], fill=color)


def draw_point(point, draw, r=10):
    show_point = []
    for p in point:
        x,y = p
        draw.ellipse((x-r, y-r, x+r, y+r), fill='green')


def run_sammed(input_image, task_type):
    # load image
    image_pil = input_image["image"].convert("RGB")
    image = np.array(image_pil)
    predictor.set_image(image)
    H,W,_ = image.shape

    scribble = np.array(input_image["mask"])
    if task_type == 'scribble_point':
        scribble = scribble.transpose(2, 1, 0)[0]
        labeled_array, num_features = ndimage.label(scribble >= 255)
        centers = ndimage.center_of_mass(scribble, labeled_array, range(1, num_features+1))
        centers = np.array(centers)
        point_coords = centers
        point_labels = np.ones(point_coords.shape[0])

        
        masks, _, _ = predictor.predict(
        point_coords=point_coords,
        point_labels=point_labels,
        mask_input = None,
        multimask_output=True 
        ) 

        mask_image = Image.new('RGBA', (H,W), color=(0, 0, 0, 0))
        mask_draw = ImageDraw.Draw(mask_image)
        for mask in masks:
            draw_mask(mask, mask_draw, random_color=True)
        image_draw = ImageDraw.Draw(image_pil)

        draw_point(point_coords,image_draw)

        image_pil = image_pil.convert('RGBA')
        image_pil.alpha_composite(mask_image)
        return [image_pil, mask_image]

    else:
        print("task_type:{} error!".format(task_type))


In [3]:
block = gr.Blocks()
block = block.queue()
with block:
    with gr.Row():
        with gr.Column():
            input_image = gr.Image(source='upload', type="pil", value="Dataset_Demo/images/amos_0507_31.png", tool="sketch",brush_radius=10)
            task_type = gr.Dropdown(["scribble_point"], value="scribble_point", label="task_type")
            run_button = gr.Button(label="Run")
        with gr.Column():
            gallery = gr.Gallery(
                label="Generated images", show_label=False, elem_id="gallery").style(preview=True, grid=2,object_fit="scale-down")

    
    with gr.Row():
        with gr.Column():
            gr.Examples(["Dataset_Demo/images/amos_0507_31.png"], inputs=input_image)


    run_button.click(fn=run_sammed, inputs=[input_image, task_type], outputs=gallery)

block.launch(debug=True, share=True, show_error=True)

C:\Users\ChengJunlong\AppData\Local\Temp\ipykernel_4760\71331735.py:10: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  gallery = gr.Gallery(
C:\Users\ChengJunlong\AppData\Local\Temp\ipykernel_4760\71331735.py:10: GradioDeprecationWarning: The 'grid' parameter will be deprecated. Please use 'grid_cols' in the constructor instead.
  gallery = gr.Gallery(


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://190277d7a434396ae8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "e:\Anacoda\envs\MMseg\lib\site-packages\gradio\routes.py", line 488, in run_predict
    output = await app.get_blocks().process_api(
  File "e:\Anacoda\envs\MMseg\lib\site-packages\gradio\blocks.py", line 1431, in process_api
    result = await self.call_function(
  File "e:\Anacoda\envs\MMseg\lib\site-packages\gradio\blocks.py", line 1109, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "e:\Anacoda\envs\MMseg\lib\site-packages\anyio\to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "e:\Anacoda\envs\MMseg\lib\site-packages\anyio\_backends\_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "e:\Anacoda\envs\MMseg\lib\site-packages\anyio\_backends\_asyncio.py", line 807, in run
    result = context.run(func, *args)
  File "e:\Anacoda\envs\MMseg\lib\site-packages\gradio\utils.py", line 706, in wrapper
    response = f(*args, **kwargs)
  Fi